# Add wine descriptions

In [43]:
# Libraries
import requests
import pandas as pd
import numpy as np
import random

In [20]:
#Download wine descriptions

# URL of the CSV file
url = 'https://huggingface.co/datasets/alfredodeza/wine-ratings/resolve/main/train.csv'

# Send a GET request to download the file
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Save the content to a CSV file
with open('train.csv', 'wb') as file:
    file.write(response.content)

# Read the CSV file into a DataFrame
descriptions = pd.read_csv('train.csv')

# Display the first few rows of the DataFrame
descriptions.head()

,name,region,variety,rating,notes
0,1000 Stories Bourbon Barrel Aged Batch Blue Ca...,"Mendocino, California",Red Wine,91.0,"This is a very special, limited release of 100..."
1,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,89.0,The California Gold Rush was a period of coura...
2,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,90.0,The California Gold Rush was a period of coura...
3,1000 Stories Bourbon Barrel Aged Zinfandel 2013,"North Coast, California",Red Wine,91.0,"The wine has a deep, rich purple color. An int..."
4,1000 Stories Bourbon Barrel Aged Zinfandel 2014,California,Red Wine,90.0,Batch #004 is the first release of the 2014 vi...


In [111]:
# Filter and cleaning
# Filter the DataFrame for 'Red Wine' or 'White Wine'
descriptions_clean = descriptions[(descriptions['variety'].isin(['Red Wine', 'White Wine']))]

# Create the USA_wine_states DataFrame
popular_states = ['California', 'Washington', 'Oregon', 'New York', 'Texas']
USA_wine_states = pd.DataFrame(popular_states, columns=['state'])

# Create a boolean mask for 'France' in 'region' column
mask_france = descriptions_clean['region'].str.contains('France', case=False, na=False)

# Create a boolean mask for USA wine states in 'region' column
mask_usa_states = descriptions_clean['region'].apply(
    lambda x: any(state in x for state in USA_wine_states['state']) if not isinstance(x, float) else False
)

# Combine the two masks
combined_mask = mask_france | mask_usa_states

# Apply the combined mask to filter the DataFrame
descriptions_clean_filtered = descriptions_clean[combined_mask]

# Remove null comments
descriptions_clean_filtered = descriptions_clean_filtered.dropna(subset=['notes'])

# Display the first few rows of the filtered DataFrame
descriptions_clean_filtered.head()

,name,region,variety,rating,notes
0,1000 Stories Bourbon Barrel Aged Batch Blue Ca...,"Mendocino, California",Red Wine,91.0,"This is a very special, limited release of 100..."
1,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,89.0,The California Gold Rush was a period of coura...
2,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,90.0,The California Gold Rush was a period of coura...
3,1000 Stories Bourbon Barrel Aged Zinfandel 2013,"North Coast, California",Red Wine,91.0,"The wine has a deep, rich purple color. An int..."
4,1000 Stories Bourbon Barrel Aged Zinfandel 2014,California,Red Wine,90.0,Batch #004 is the first release of the 2014 vi...


In [118]:
# Add deciles
# Function to calculate deciles without dropping duplicates
def calculate_decile(df, column):
    sorted_df = df.sort_values(by=column)  # Sort DataFrame by 'rating' column
    num_rows = len(sorted_df)
    deciles = pd.Series([i for i in range(1, 11) for _ in range(num_rows // 10)], index=sorted_df.index[:num_rows // 10 * 10])  # Assign deciles
    # Assign the remaining rows to the last decile if the number of rows is not evenly divisible by 10
    deciles = pd.concat([deciles, pd.Series([10] * (num_rows % 10), index=sorted_df.index[num_rows // 10 * 10:])])
    return deciles

# Calculate deciles for 'Red Wine'
red_wine_indices = descriptions_clean_filtered['variety'] == 'Red Wine'
descriptions_clean_filtered.loc[red_wine_indices, 'decile_rating'] = \
    calculate_decile(descriptions_clean_filtered[red_wine_indices], 'rating')

# Calculate deciles for 'White Wine'
white_wine_indices = descriptions_clean_filtered['variety'] == 'White Wine'
descriptions_clean_filtered.loc[white_wine_indices, 'decile_rating'] = \
    calculate_decile(descriptions_clean_filtered[white_wine_indices], 'rating')

# Display the first few rows of the DataFrame to check the new column
descriptions_clean_filtered.head()

,name,region,variety,rating,notes,decile_rating
0,1000 Stories Bourbon Barrel Aged Batch Blue Ca...,"Mendocino, California",Red Wine,91.0,"This is a very special, limited release of 100...",5.0
1,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,89.0,The California Gold Rush was a period of coura...,1.0
2,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,90.0,The California Gold Rush was a period of coura...,2.0
3,1000 Stories Bourbon Barrel Aged Zinfandel 2013,"North Coast, California",Red Wine,91.0,"The wine has a deep, rich purple color. An int...",6.0
4,1000 Stories Bourbon Barrel Aged Zinfandel 2014,California,Red Wine,90.0,Batch #004 is the first release of the 2014 vi...,2.0


In [120]:
# Replace 'Red Wine' with 'red' and 'White Wine' with 'white' in the 'variety' column
descriptions_clean_filtered['variety'] = descriptions_clean_filtered['variety'].replace({'Red Wine': 'red', 'White Wine': 'white'})

# Rename the 'variety' column to 'flight' (assuming it's the column to be renamed)
descriptions_clean_filtered = descriptions_clean_filtered.rename(columns={'variety': 'flight'})

# Display the updated DataFrame to verify the changes
descriptions_clean_filtered.head()

,name,region,flight,rating,notes,decile_rating
0,1000 Stories Bourbon Barrel Aged Batch Blue Ca...,"Mendocino, California",red,91.0,"This is a very special, limited release of 100...",5.0
1,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,red,89.0,The California Gold Rush was a period of coura...,1.0
2,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,red,90.0,The California Gold Rush was a period of coura...,2.0
3,1000 Stories Bourbon Barrel Aged Zinfandel 2013,"North Coast, California",red,91.0,"The wine has a deep, rich purple color. An int...",6.0
4,1000 Stories Bourbon Barrel Aged Zinfandel 2014,California,red,90.0,Batch #004 is the first release of the 2014 vi...,2.0


In [123]:
# Read 'wines_2012.csv'
# Read the CSV file into a DataFrame named 'wines'
wines = pd.read_csv('wines_2012.csv')

# Grouping 'wines' by 'wine' column and calculating the average score
average_scores = wines.groupby(['flight','wine'])['score'].mean().reset_index()

# Display the new DataFrame
average_scores

,flight,wine,score
0,red,A2,14.500000
1,red,B2,15.833333
2,red,C2,13.111111
3,red,D2,15.000000
4,red,E2,14.555556
5,red,F2,14.111111
6,red,G2,14.222222
7,red,H2,13.611111
8,red,I2,11.666667
9,red,J2,15.333333


In [124]:
# Create a function to calculate deciles
def calculate_decile(df, column):
    return pd.qcut(df[column], 10, labels=False) + 1

# Calculate deciles for 'flight' = 'red'
average_scores.loc[average_scores['flight'] == 'red', 'decile_rating'] = \
    calculate_decile(average_scores[average_scores['flight'] == 'red'], 'score')

# Calculate deciles for 'flight' = 'white'
average_scores.loc[average_scores['flight'] == 'white', 'decile_rating'] = \
    calculate_decile(average_scores[average_scores['flight'] == 'white'], 'score')

# Display the DataFrame to check the new column
average_scores


,flight,wine,score,decile_rating
0,red,A2,14.500000,6.0
1,red,B2,15.833333,10.0
2,red,C2,13.111111,2.0
3,red,D2,15.000000,8.0
4,red,E2,14.555556,7.0
5,red,F2,14.111111,4.0
6,red,G2,14.222222,5.0
7,red,H2,13.611111,3.0
8,red,I2,11.666667,1.0
9,red,J2,15.333333,9.0


In [125]:
# Perform left join on 'flight' and 'decile_rating' columns
merged_df = pd.merge(average_scores, descriptions_clean_filtered, on=['flight', 'decile_rating'], how='left')
merged_df

,flight,wine,score,decile_rating,name,region,rating,notes
0,red,A2,14.500000,6.0,1000 Stories Bourbon Barrel Aged Zinfandel 2013,"North Coast, California",91.0,"The wine has a deep, rich purple color. An int..."
1,red,A2,14.500000,6.0,1000 Stories Bourbon Barrel Aged Zinfandel 2016,California,91.0,"1,000 Stories Bourbon barrel-aged Zinfandel is..."
2,red,A2,14.500000,6.0,1849 Declaration Napa Valley Cabernet Sauvigno...,"Napa Valley, California",91.0,The palate is robust with flavors of dark blue...
3,red,A2,14.500000,6.0,689 Cellars Killer Drop Red Blend 2014,California,91.0,Our 2014 release of KILLER DROP brings togethe...
4,red,A2,14.500000,6.0,A Tribute to Grace Santa Barbara County Grenac...,"Santa Barbara, Central Coast, California",91.0,The 2016 Santa Barbara County Grenache shows a...
...,...,...,...,...,...,...,...,...
20001,white,J1,13.722222,2.0,L'Ecole 41 Semillon 2015,"Columbia Valley, Washington",89.0,"Rich with vibrant fruit and balanced acidity, ..."
20002,white,J1,13.722222,2.0,Leese-Fitch Sauvignon Blanc 2015,California,89.0,"Zesty and refreshing, this Sauvignon Blanc has..."
20003,white,J1,13.722222,2.0,Leese-Fitch Sauvignon Blanc 2017,California,89.0,2017 was a good vintage for Sauvignon Blanc. M...
20004,white,J1,13.722222,2.0,Lemelson Dry Riesling 2015,"Dundee Hills, Willamette Valley, Oregon",89.0,To help maximize aromatic intensity and comple...


In [134]:
# Only select one description per wine
## Group by 'wine' and select one random record from each group
np.random.seed(1304)
random_records = merged_df.groupby('wine').apply(lambda x: x.sample(1))

# Reset the index of the resulting DataFrame
random_records.reset_index(drop=True, inplace=True)
random_records

/tmp/ipykernel_20341/2624778012.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  random_records = merged_df.groupby('wine').apply(lambda x: x.sample(1))


,flight,wine,score,decile_rating,name,region,rating,notes
0,white,A1,14.611111,8.0,Au Bon Climat Sanford and Benedict Chardonnay ...,"Central Coast, California",92.0,"After hand-picking, whole cluster pressing, an..."
1,red,A2,14.500000,6.0,Adelaida End Post Cabernet Sauvignon 2015,"Paso Robles, Central Coast, California",91.0,"A delicious bouquet of cassis syrup, violets, ..."
2,white,B1,15.000000,9.0,Duckhorn Sauvignon Blanc 2010,"Napa Valley, California",93.0,This enticing Sauvignon Blanc offers a beautif...
3,red,B2,15.833333,10.0,Corliss Cabernet Sauvignon 2009,"Columbia Valley, Washington",95.0,"Brooding dark fruit aromas of wild fig, blackb..."
4,white,C1,13.833333,4.0,Fogarty Monterey Guwurztraminer 2013,"Central Coast, California",90.0,Our 2013 Gewurztraminer displays classic rose ...
5,red,C2,13.111111,2.0,Chateau d'Issan Blason d'Issan (Futures Pre-Sa...,"Margaux, Bordeaux, France",90.0,"Blend: 60% Merlot, 40% Cabernet Sauvignon"
6,white,D1,15.055556,10.0,Guigal Ermitage Ex Voto Blanc 2005,"Hermitage, Rhone, France",97.0,"""Light gold. A stunning bouquet offers powerfu..."
7,red,D2,15.000000,8.0,Domaine du Vieux Lazaret Chateauneuf-du-Pape C...,"Chateauneuf-du-Pape, Rhone, France",93.0,"Deep garnet color. Notes of red cherries, plum..."
8,white,E1,14.444444,7.0,Chalk Hill Chardonnay 1998,"Sonoma County, California",91.0,This is perhaps the most elegant Chardonnay ev...
9,red,E2,14.555556,7.0,Clos Du Val Estate Cabernet Sauvignon 2014,"Napa Valley, California",92.0,This wine delivers everything one is looking f...


In [138]:
# Finally, assign the description to the original dataframe wines
# Left join 'wines' with 'random_records' on 'wine' column and only add the 'notes' column to 'wines'
wines_with_description = pd.merge(wines, random_records[['wine', 'notes']], on='wine', how='left')

# Rename the 'notes' column to 'description'
wines_with_description.rename(columns={'notes': 'description'}, inplace=True)
wines_with_description.to_csv('wines_with_description.csv', index=False)
wines_with_description

,judge,flight,wine,score,wine.amer,judge.amer,description
0,Jean-M Cardebat,white,A1,10.0,1,0,"After hand-picking, whole cluster pressing, an..."
1,Jean-M Cardebat,white,B1,13.0,1,0,This enticing Sauvignon Blanc offers a beautif...
2,Jean-M Cardebat,white,C1,14.0,0,0,Our 2013 Gewurztraminer displays classic rose ...
3,Jean-M Cardebat,white,D1,15.0,0,0,"""Light gold. A stunning bouquet offers powerfu..."
4,Jean-M Cardebat,white,E1,8.0,1,0,This is perhaps the most elegant Chardonnay ev...
...,...,...,...,...,...,...,...
175,Francis Schott,red,F2,12.0,1,1,A cascade of aromatics starts with a bright fl...
176,Francis Schott,red,G2,15.0,0,1,We named this wine in homage of a wonderful re...
177,Francis Schott,red,H2,16.0,1,1,The 1999 vintage was harvested a few days late...
178,Francis Schott,red,I2,7.0,1,1,"Wonderful berry fruit, subtle, chalky tannins ..."
